In [2]:
import tensorflow as tf

In [7]:
x=tf.Variable(3,name='x')
y=tf.Variable(4,name='y')
f=x*x*y+y+2

In [8]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)
sess.close()

42


In [12]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()
    print(result)


42


In [13]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result=f.eval()
    print(result)

42


In [18]:
sess=tf.InteractiveSession()
init.run()
result=f.eval()
print(result)
sess.close()

42


In [19]:
x1=tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [21]:
graph=tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)

In [22]:
x2.graph is graph

True

In [23]:
x2.graph is tf.get_default_graph()

False

In [24]:
w=tf.constant(3)
x=w+2
y=x+5
z=x*3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [25]:
with tf.Session() as sess:
    y_val,z_val=sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


In [4]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()
m,n=housing.data.shape

housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
XT=tf.transpose(X)

theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_val=theta.eval()

print(theta_val)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [27]:
X=housing_data_plus_bias
y=housing.target.reshape(-1,1)
theta_numpy=np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
print(theta_numpy)

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


In [29]:
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg.fit(housing.data,housing.target.reshape(-1,1))
print(np.r_[lin_reg.intercept_.reshape(-1,1),lin_reg.coef_.T])#将linearRegression 中的截距项与参数按照行堆叠起来

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


In [31]:
from sklearn.preprocessing import StandardScaler

housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
scaler=StandardScaler()
# scaler.fit(housing_data_plus_bias)
# scaled_housing_data_plus_bias=scaler.transform(housing_data_plus_bias)
scaled_housing_data_plus_bias=scaler.fit_transform(housing_data_plus_bias)  #fit()+transform()=fit_transform()
print(scaled_housing_data_plus_bias[:2])
#应该先进行正则变换,再进行加上bias

scaled_housing_data=scaler.fit(housing.data).transform(housing.data)
scaled_housing_data_plus_bias=np.c_[np.ones((m,1)),scaled_housing_data]
print(scaled_housing_data_plus_bias[:2])
scaled_housing_data=scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias=np.c_[np.ones((m,1)),scaled_housing_data]
print(scaled_housing_data_plus_bias[:2])


[[ 0.          2.34476576  0.98214266  0.62855945 -0.15375759 -0.9744286
  -0.04959654  1.05254828 -1.32783522]
 [ 0.          2.33223796 -0.60701891  0.32704136 -0.26333577  0.86143887
  -0.09251223  1.04318455 -1.32284391]]
[[ 1.          2.34476576  0.98214266  0.62855945 -0.15375759 -0.9744286
  -0.04959654  1.05254828 -1.32783522]
 [ 1.          2.33223796 -0.60701891  0.32704136 -0.26333577  0.86143887
  -0.09251223  1.04318455 -1.32284391]]
[[ 1.          2.34476576  0.98214266  0.62855945 -0.15375759 -0.9744286
  -0.04959654  1.05254828 -1.32783522]
 [ 1.          2.33223796 -0.60701891  0.32704136 -0.26333577  0.86143887
  -0.09251223  1.04318455 -1.32284391]]


In [32]:
n_epoches=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name='mse')
gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoches):
        if epoch %100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()
print(best_theta)

Epoch 0 MSE= 2.75443
Epoch 100 MSE= 0.632222
Epoch 200 MSE= 0.57278
Epoch 300 MSE= 0.558501
Epoch 400 MSE= 0.549069
Epoch 500 MSE= 0.542288
Epoch 600 MSE= 0.537379
Epoch 700 MSE= 0.533822
Epoch 800 MSE= 0.531243
Epoch 900 MSE= 0.529371
[[  2.06855226e+00]
 [  7.74078071e-01]
 [  1.31192386e-01]
 [ -1.17845096e-01]
 [  1.64778158e-01]
 [  7.44080753e-04]
 [ -3.91945168e-02]
 [ -8.61356616e-01]
 [ -8.23479712e-01]]


In [37]:
tf.reset_default_graph()
n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")


In [39]:
gradients=tf.gradients(mse,[theta])[0]


In [42]:
training_op=tf.assign(theta,theta-learning_rate*gradients)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoches):
        if epoch %100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

print("Best theta:")
print(best_theta)

Epoch 0 MSE= 2.75443
Epoch 100 MSE= 0.632222
Epoch 200 MSE= 0.57278
Epoch 300 MSE= 0.558501
Epoch 400 MSE= 0.549069
Epoch 500 MSE= 0.542288
Epoch 600 MSE= 0.537379
Epoch 700 MSE= 0.533822
Epoch 800 MSE= 0.531243
Epoch 900 MSE= 0.529371
Best theta:
[[  2.06855249e+00]
 [  7.74078071e-01]
 [  1.31192386e-01]
 [ -1.17845066e-01]
 [  1.64778143e-01]
 [  7.44078017e-04]
 [ -3.91945094e-02]
 [ -8.61356676e-01]
 [ -8.23479772e-01]]


In [45]:
tf.reset_default_graph()
n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoches):
        if epoch%100==0:
            print("Epoch",epoch,"MSE =",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.75443
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.57278
Epoch 300 MSE = 0.558501
Epoch 400 MSE = 0.549069
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.537379
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.531243
Epoch 900 MSE = 0.529371
Best theta:
[[  2.06855249e+00]
 [  7.74078071e-01]
 [  1.31192386e-01]
 [ -1.17845066e-01]
 [  1.64778143e-01]
 [  7.44078017e-04]
 [ -3.91945094e-02]
 [ -8.61356676e-01]
 [ -8.23479772e-01]]


In [47]:
tf.reset_default_graph()
n_epochs=1000
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred=tf.matmul(X,theta,name="predictions")
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epoches):
        if epoch%100==0:
            print("Epoch",epoch,"MSE =",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()
print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.75443
Epoch 100 MSE = 0.527316
Epoch 200 MSE = 0.524414
Epoch 300 MSE = 0.524328
Epoch 400 MSE = 0.524322
Epoch 500 MSE = 0.524321
Epoch 600 MSE = 0.524321
Epoch 700 MSE = 0.52432
Epoch 800 MSE = 0.524321
Epoch 900 MSE = 0.524321
Best theta:
[[ 2.06855798]
 [ 0.82961673]
 [ 0.11875112]
 [-0.26552212]
 [ 0.30569226]
 [-0.00450316]
 [-0.03932616]
 [-0.89989167]
 [-0.87054664]]


In [50]:
tf.reset_default_graph()
A=tf.placeholder(tf.float32,shape=(None,3))
B=A+5
with tf.Session() as sess:
    B_eval_1=B.eval(feed_dict={A:[[1,2,3]]})
    B_eval_2=B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})
print(B_eval_2)

[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [12]:
tf.reset_default_graph()
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
scaler=StandardScaler()

scaled_housing_data=scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias=np.c_[np.ones((m,1)),scaled_housing_data]
X=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="y")

theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred=tf.matmul(X,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
learning_rate=0.01
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()

n_epochs=10
batch_size=100

n_batches=int(np.ceil(m/batch_size))

def fetch_batch(epoch,batch_index,batch_size):
    np.random.seed(epoch*n_batches+batch_index)
    indices=np.random.randint(m,size=batch_size)
    X_batch=scaled_housing_data_plus_bias[indices]
    y_batch=housing.target.reshape(-1,1)[indices]
    return X_batch,y_batch

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
    best_theta=theta.eval()

print(best_theta)


[[ 2.07001591]
 [ 0.82045609]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.31134021]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.87950081]]


# 模型的保存与加载

In [5]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
housing=fetch_california_housing()
m,n=housing.data.shape
scaler=StandardScaler()
scaled_housing_data=scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias=np.c_[np.ones((m,1)),scaled_housing_data]
learning_rate=0.01
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0,seed=42),name="theta")
y_pred=tf.matmul(X,theta)
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name="mse")
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)
init=tf.global_variables_initializer()
n_epochs=1000
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch=",epoch,"MSE=",mse.eval())
            save_path=saver.save(sess,"./tmp/my_model.ckpt")
        sess.run(training_op)
    best_theta=theta.eval()
    save_path=saver.save(sess,"./tmp/my_model_final.ckpt")
print(best_theta)

Epoch= 0 MSE= 2.75443
Epoch= 100 MSE= 0.632222
Epoch= 200 MSE= 0.57278
Epoch= 300 MSE= 0.558501
Epoch= 400 MSE= 0.549069
Epoch= 500 MSE= 0.542288
Epoch= 600 MSE= 0.537379
Epoch= 700 MSE= 0.533822
Epoch= 800 MSE= 0.531243
Epoch= 900 MSE= 0.529371
[[  2.06855249e+00]
 [  7.74078071e-01]
 [  1.31192386e-01]
 [ -1.17845066e-01]
 [  1.64778143e-01]
 [  7.44078017e-04]
 [ -3.91945094e-02]
 [ -8.61356676e-01]
 [ -8.23479772e-01]]


In [14]:
best_theta

array([[  2.06855249e+00],
       [  7.74078071e-01],
       [  1.31192386e-01],
       [ -1.17845066e-01],
       [  1.64778143e-01],
       [  7.44078017e-04],
       [ -3.91945094e-02],
       [ -8.61356676e-01],
       [ -8.23479772e-01]], dtype=float32)

In [4]:

with tf.Session() as sess:
    saver.restore(sess,"./tmp/my_model_final.ckpt")
    best_theta_restore=theta.eval()
print(best_theta_restore)

INFO:tensorflow:Restoring parameters from ./tmp/my_model_final.ckpt
[[  2.06855249e+00]
 [  7.74078071e-01]
 [  1.31192386e-01]
 [ -1.17845066e-01]
 [  1.64778143e-01]
 [  7.44078017e-04]
 [ -3.91945094e-02]
 [ -8.61356676e-01]
 [ -8.23479772e-01]]


# graph 的可视化

In [4]:
tf.reset_default_graph()

